# Data Wrangling: Join, Combine, and Reshape

## 1. Hierarchical Indexing
- index.names / column.names : hierarchical indexing에서는 여러개의 index 및 column 지정이 가능하다

In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.Series(np.random.randn(9),
 ...: index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],
 ...: [1, 2, 3, 1, 3, 1, 2, 2, 3]])

In [3]:
data

a  1   -0.689749
   2   -0.699501
   3    0.939147
b  1    1.270844
   3    1.368187
c  1    0.723994
   2    0.128186
d  2    0.352112
   3   -1.275445
dtype: float64

In [4]:
data['b']

1    1.270844
3    1.368187
dtype: float64

In [5]:
data.loc[['b', 'd']]

b  1    1.270844
   3    1.368187
d  2    0.352112
   3   -1.275445
dtype: float64

In [6]:
data.unstack() #unstack : 데이터 펼치기

,1,2,3
a,-0.689749,-0.699501,0.939147
b,1.270844,NaN,1.368187
c,0.723994,0.128186,NaN
d,NaN,0.352112,-1.275445


In [7]:
frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
 ....: index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
 ....: columns=[['Ohio', 'Ohio', 'Colorado'],
 ....: ['Green', 'Red', 'Green']])

In [8]:
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [9]:
frame.index.names = ['key1', 'key2']

In [10]:
frame.columns.names = ['state', 'color']

In [11]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

### Reordering and Sorting Levels
- level : multi-index에서 index와 column을 지정할 때 ★
- pd.swaplevel : 지정한 level을 서로 바꾸기

In [12]:
frame.swaplevel('key1', 'key2')

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [13]:
frame.sort_index(level=1)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [14]:
frame.swaplevel(0,1).sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

### Summary Statistics by Level

In [15]:
frame.sum(level='key2')

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [16]:
frame.sum(level='color', axis=1)

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

### Indexing with a DataFrame’s columns
- pd.set_index([]) : 현재 있는 dataframe의 column 중에 특정한 column을 index로 지정하고 싶을 때 (drop=False : index로 지정하되 원 데이터를 그대로 두고싶을 때)
- pd.reset_index() : 현재의 index를 index가 아닌 데이터로 지정하고 싶을때

In [17]:
frame = pd.DataFrame({'a': range(7), 'b': range(7, 0, -1),
 ....: 'c': ['one', 'one', 'one', 'two', 'two',
 ....: 'two', 'two'],
 ....: 'd': [0, 1, 2, 0, 1, 2, 3]})

In [18]:
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [19]:
frame2 = frame.set_index(['c', 'd'])

In [20]:
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [21]:
frame.set_index(['c', 'd'], drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

In [22]:
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


## 2. Combining and Merging Datasets

### Database-Style DataFrame Joins
- pd.merge(df1, df2, how='inner') : 열을 기준으로, 두 데이터 프레임에 모두 있는 key만을 가져와 합침. 만약에 key 이름이 다르면 left_on / right_on 으로 key를 지정해주어야함
- how='outer' : 공통 여부에 상관없이 두 데이터를 모두 합침 / how='left' : 왼쪽에만 있는 key를 기준으로 합침
- suffixes = (a, b) : column에 왼쪽 데이터는 a를, 오른쪽 데이터는 b인 접미사를 붙임 // default값 : ('_x', '_y')

In [24]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
 ....: 'data1': range(7)})

In [25]:
df2 = pd.DataFrame({'key': ['a', 'b', 'd'],
 ....: 'data2': range(3)})

In [26]:
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [27]:
df2

,key,data2
0,a,0
1,b,1
2,d,2


In [28]:
pd.merge(df1, df2)

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [29]:
pd.merge(df1, df2, on='key')

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [30]:
df3 = pd.DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
 ....: 'data1': range(7)})

In [31]:
df4 = pd.DataFrame({'rkey': ['a', 'b', 'd'],
 ....: 'data2': range(3)})

In [32]:
pd.merge(df3, df4, left_on='lkey', right_on='rkey')

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


In [33]:
pd.merge(df1, df2, how='outer')

,key,data1,data2
0,b,0.0,1.0
1,b,1.0,1.0
2,b,6.0,1.0
3,a,2.0,0.0
4,a,4.0,0.0
5,a,5.0,0.0
6,c,3.0,NaN
7,d,NaN,2.0


### Merging on Index
- merge를 이용하여 index를 쓰고싶으면 옵션으로 left_index=True 나 right_index=True를 써야함
- 둘다 index로 합치고 싶으면 left_index=True와 right_index=True 둘다 사용
- df1.join(df2) -> df1와 df2를 index를 이용하여 합침 //기본값: how='left'

In [34]:
left1 = pd.DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'],
 ....: 'value': range(6)})

In [35]:
right1 = pd.DataFrame({'group_val': [3.5, 7]}, index=['a', 'b'])

In [36]:
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [37]:
right1

,group_val
a,3.5
b,7.0


In [38]:
pd.merge(left1, right1, left_on='key', right_index=True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


In [39]:
pd.merge(left1, right1, left_on='key', right_index=True, how='outer')

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


In [40]:
lefth = pd.DataFrame({'key1': ['Ohio', 'Ohio', 'Ohio',
 ....: 'Nevada', 'Nevada'],
 ....: 'key2': [2000, 2001, 2002, 2001, 2002],
 ....: 'data': np.arange(5.)})

In [41]:
righth = pd.DataFrame(np.arange(12).reshape((6, 2)),
 ....: index=[['Nevada', 'Nevada', 'Ohio', 'Ohio',
 ....: 'Ohio', 'Ohio'],
 ....: [2001, 2000, 2000, 2000, 2001, 2002]],
 ....: columns=['event1', 'event2'])

In [42]:
lefth

,key1,key2,data
0,Ohio,2000,0.0
1,Ohio,2001,1.0
2,Ohio,2002,2.0
3,Nevada,2001,3.0
4,Nevada,2002,4.0


In [43]:
righth

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

In [44]:
pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True)

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4,5
0,Ohio,2000,0.0,6,7
1,Ohio,2001,1.0,8,9
2,Ohio,2002,2.0,10,11
3,Nevada,2001,3.0,0,1


In [45]:
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]],
 ....: index=['a', 'c', 'e'],
 ....: columns=['Ohio', 'Nevada'])

In [46]:
right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],
 ....: index=['b', 'c', 'd', 'e'],
 ....: columns=['Missouri', 'Alabama'])

In [47]:
left2

,Ohio,Nevada
a,1.0,2.0
c,3.0,4.0
e,5.0,6.0


In [48]:
right2

,Missouri,Alabama
b,7.0,8.0
c,9.0,10.0
d,11.0,12.0
e,13.0,14.0


In [49]:
pd.merge(left2, right2, left_index=True, right_index=True)

,Ohio,Nevada,Missouri,Alabama
c,3.0,4.0,9.0,10.0
e,5.0,6.0,13.0,14.0


In [50]:
left2.join(right2)

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
c,3.0,4.0,9.0,10.0
e,5.0,6.0,13.0,14.0


In [51]:
left1.join(right1, on='key')

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


In [52]:
another = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [16., 17.]],
 ....: index=['a', 'c', 'e', 'f'],
 ....: columns=['New York', 'Oregon'])

In [53]:
left2.join([right2, another])

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0,9.0,10.0
e,5.0,6.0,13.0,14.0,11.0,12.0


### Concatenating Along an Axis
- np.concatenate(axis=0)
- pd.concat(axis=0, how='outer') // sort=True로 정렬해주어야 오류가 없음
- join_axes= 옵션으로 join할 축들 지정 가능함
- keys= 옵션으로 hierarchical index 만들기 가능! axis=0이면 인덱스 axis=1이면 column
- names= level의 이름
- ignore_index=True : 원래의 index 쓰지 않기

In [54]:
arr = np.arange(12).reshape((3, 4))

In [55]:
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [56]:
np.concatenate([arr, arr], axis=1)

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

In [57]:
s1 = pd.Series([0, 1], index=['a', 'b']);
s2 = pd.Series([2, 3, 4], index=['c', 'd', 'e']);
s3 = pd.Series([5, 6], index=['f', 'g']);

In [58]:
pd.concat([s1, s2, s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

In [59]:
s4 = pd.concat([s1, s3])

In [61]:
s4

a    0
b    1
f    5
g    6
dtype: int64

In [65]:
pd.concat([s1, s4], axis=1, sort=True)

,0,1
a,0.0,0
b,1.0,1
f,NaN,5
g,NaN,6


In [67]:
pd.concat([s1, s4], axis=1, join='inner')

,0,1
a,0,0
b,1,1


In [68]:
pd.concat([s1, s4], axis=1, join_axes=[['a', 'c', 'b', 'e']])

,0,1
a,0.0,0.0
c,NaN,NaN
b,1.0,1.0
e,NaN,NaN


In [69]:
result = pd.concat([s1, s1, s3], keys=['one', 'two', 'three'])

In [70]:
result

one    a    0
       b    1
two    a    0
       b    1
three  f    5
       g    6
dtype: int64

In [71]:
df1 = pd.DataFrame(np.arange(6).reshape(3, 2), index=['a', 'b', 'c'],
 ....: columns=['one', 'two'])

In [72]:
df2 = pd.DataFrame(5 + np.arange(4).reshape(2, 2), index=['a', 'c'],
 ....: columns=['three', 'four'])

In [73]:
df1

,one,two
a,0,1
b,2,3
c,4,5


In [74]:
df2

,three,four
a,5,6
c,7,8


In [76]:
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'], sort=True)

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [78]:
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'], names=['upper', 'lower'], sort=True)

upper level1     level2     
lower    one two  three four
a          0   1    5.0  6.0
b          2   3    NaN  NaN
c          4   5    7.0  8.0

In [79]:
df1 = pd.DataFrame(np.random.randn(3, 4), columns=['a', 'b', 'c', 'd'])

In [80]:
df2 = pd.DataFrame(np.random.randn(2, 3), columns=['b', 'd', 'a'])

In [81]:
df1

,a,b,c,d
0,-0.488712,0.590137,0.169332,-1.567686
1,-0.878402,-0.882066,1.096140,0.540428
2,0.021963,-0.052741,1.354538,-0.168475


In [82]:
df2

,b,d,a
0,1.515993,-0.143357,-1.086537
1,-0.510018,-0.187238,0.422901


In [84]:
pd.concat([df1, df2], sort=True)

,a,b,c,d
0,-0.488712,0.590137,0.169332,-1.567686
1,-0.878402,-0.882066,1.096140,0.540428
2,0.021963,-0.052741,1.354538,-0.168475
0,-1.086537,1.515993,NaN,-0.143357
1,0.422901,-0.510018,NaN,-0.187238


In [85]:
pd.concat([df1, df2], sort=True, ignore_index=True)

,a,b,c,d
0,-0.488712,0.590137,0.169332,-1.567686
1,-0.878402,-0.882066,1.096140,0.540428
2,0.021963,-0.052741,1.354538,-0.168475
3,-1.086537,1.515993,NaN,-0.143357
4,0.422901,-0.510018,NaN,-0.187238


### Combining Data with Overlap
- combine_first : 원래 있는 데이터는 왼쪽값을, NaN값은 오른쪽으로 채움

In [86]:
a = pd.Series([np.nan, 2.5, np.nan, 3.5, 4.5, np.nan],
 .....: index=['f', 'e', 'd', 'c', 'b', 'a'])

In [87]:
b = pd.Series(np.arange(len(a), dtype=np.float64),
 .....: index=['f', 'e', 'd', 'c', 'b', 'a'])

In [88]:
b[-1] = np.nan

In [89]:
a

f    NaN
e    2.5
d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64

In [90]:
b

f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    NaN
dtype: float64

In [92]:
np.where(pd.isnull(a), b, a) #np.where(조건문, True일때 값, False일 때 값)

array([0. , 2.5, 2. , 3.5, 4.5, nan])

In [94]:
b[:-2]

f    0.0
e    1.0
d    2.0
c    3.0
dtype: float64

In [95]:
a[2:]

d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64

In [93]:
b[:-2].combine_first(a[2:])

a    NaN
b    4.5
c    3.0
d    2.0
e    1.0
f    0.0
dtype: float64

In [96]:
df1 = pd.DataFrame({'a': [1., np.nan, 5., np.nan],
 .....: 'b': [np.nan, 2., np.nan, 6.],
 .....: 'c': range(2, 18, 4)})

In [97]:
df2 = pd.DataFrame({'a': [5., 4., np.nan, 3., 7.],
 .....: 'b': [np.nan, 3., 4., 6., 8.]})

In [99]:
df1

,a,b,c
0,1.0,NaN,2
1,NaN,2.0,6
2,5.0,NaN,10
3,NaN,6.0,14


In [100]:
df2

,a,b
0,5.0,NaN
1,4.0,3.0
2,NaN,4.0
3,3.0,6.0
4,7.0,8.0


In [101]:
df1.combine_first(df2)

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


## 3. Reshaping and Pivoting

### Reshaping with Hierarchical Indexing
- pd.stack(axis=1) / pd.unstack(axis=1) : 쌓기 / 펼치기
- 쌓을때는 NaN값을 제외하고, 펼칠때는 NaN값을 포함하는데 쌓을때 NaN값을 포함하고 싶으면 dropna=False 옵션 사용

In [102]:
data = pd.DataFrame(np.arange(6).reshape((2, 3)),
 .....: index=pd.Index(['Ohio', 'Colorado'], name='state'),
 .....: columns=pd.Index(['one', 'two', 'three'],
 .....: name='number'))

In [103]:
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [105]:
result = data.stack()

In [106]:
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int32

In [107]:
result.unstack(0)

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [108]:
result.unstack('state')

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [109]:
s1 = pd.Series([0, 1, 2, 3], index=['a', 'b', 'c', 'd'])

In [110]:
s2 = pd.Series([4, 5, 6], index=['c', 'd', 'e'])

In [112]:
s1

a    0
b    1
c    2
d    3
dtype: int64

In [113]:
s2

c    4
d    5
e    6
dtype: int64

In [111]:
data2 = pd.concat([s1, s2], keys=['one', 'two'])

In [115]:
data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: int64

In [114]:
data2.unstack()

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


In [117]:
data2.unstack().stack() #stack은 NaN값이 포함된 행은 제외한다.

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
two  c    4.0
     d    5.0
     e    6.0
dtype: float64

In [116]:
data2.unstack().stack(dropna=False)

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
     e    NaN
two  a    NaN
     b    NaN
     c    4.0
     d    5.0
     e    6.0
dtype: float64

In [118]:
df = pd.DataFrame({'left': result, 'right': result + 5},
 .....: columns=pd.Index(['left', 'right'], name='side'))

In [119]:
df

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [120]:
df.unstack('state')

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

In [121]:
df.unstack('state').stack('side')

state         Colorado  Ohio
number side                 
one    left          3     0
       right         8     5
two    left          4     1
       right         9     6
three  left          5     2
       right        10     7

### Pivoting “Long” to “Wide” Format
- pivot table(행 인덱스로 사용할 열 이름, 열 인덱스로 사용할 열 이름, 데이터로 사용할 열 이름) // pivot table은 열 중 두개를 key로 사용하여 만들어진 table이다

In [122]:
data = pd.read_csv('examples/macrodata.csv')

In [123]:
data.head()

,year,quarter,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
0,1959.0,1.0,2710.349,1707.4,286.898,470.045,1886.9,28.98,139.7,2.82,5.8,177.146,0.00,0.00
1,1959.0,2.0,2778.801,1733.7,310.859,481.301,1919.7,29.15,141.7,3.08,5.1,177.830,2.34,0.74
2,1959.0,3.0,2775.488,1751.8,289.226,491.260,1916.4,29.35,140.5,3.82,5.3,178.657,2.74,1.09
3,1959.0,4.0,2785.204,1753.7,299.356,484.052,1931.3,29.37,140.0,4.33,5.6,179.386,0.27,4.06
4,1960.0,1.0,2847.699,1770.5,331.722,462.199,1955.5,29.54,139.6,3.50,5.2,180.007,2.31,1.19


In [124]:
periods = pd.PeriodIndex(year=data.year, quarter=data.quarter,
 .....: name='date')

In [125]:
columns = pd.Index(['realgdp', 'infl', 'unemp'], name='item')

In [126]:
data = data.reindex(columns=columns)

In [131]:
data[:5]

item,realgdp,infl,unemp
date,,,
1959-03-31,2710.349,0.00,5.8
1959-06-30,2778.801,2.34,5.1
1959-09-30,2775.488,2.74,5.3
1959-12-31,2785.204,0.27,5.6
1960-03-31,2847.699,2.31,5.2


In [127]:
data.index = periods.to_timestamp('D', 'end')
#to_timestamp : 문자열을 시간을 나타내는 timestamp로 바꿔줌

In [132]:
data.stack()

date        item   
1959-03-31  realgdp     2710.349
            infl           0.000
            unemp          5.800
1959-06-30  realgdp     2778.801
            infl           2.340
            unemp          5.100
1959-09-30  realgdp     2775.488
            infl           2.740
            unemp          5.300
1959-12-31  realgdp     2785.204
            infl           0.270
            unemp          5.600
1960-03-31  realgdp     2847.699
            infl           2.310
            unemp          5.200
1960-06-30  realgdp     2834.390
            infl           0.140
            unemp          5.200
1960-09-30  realgdp     2839.022
            infl           2.700
            unemp          5.600
1960-12-31  realgdp     2802.616
            infl           1.210
            unemp          6.300
1961-03-31  realgdp     2819.264
            infl          -0.400
            unemp          6.800
1961-06-30  realgdp     2872.005
            infl           1.470
            unemp      

In [128]:
ldata = data.stack().reset_index().rename(columns={0:'value'})
#reset_index : index를 일반자료형으로 바꾸는 기능

In [129]:
ldata[:10]

,date,item,value
0,1959-03-31,realgdp,2710.349
1,1959-03-31,infl,0.000
2,1959-03-31,unemp,5.800
3,1959-06-30,realgdp,2778.801
4,1959-06-30,infl,2.340
5,1959-06-30,unemp,5.100
6,1959-09-30,realgdp,2775.488
7,1959-09-30,infl,2.740
8,1959-09-30,unemp,5.300
9,1959-12-31,realgdp,2785.204


In [133]:
pivoted = ldata.pivot('date', 'item', 'value')

In [134]:
pivoted[:10]

item,infl,realgdp,unemp
date,,,
1959-03-31,0.00,2710.349,5.8
1959-06-30,2.34,2778.801,5.1
1959-09-30,2.74,2775.488,5.3
1959-12-31,0.27,2785.204,5.6
1960-03-31,2.31,2847.699,5.2
1960-06-30,0.14,2834.390,5.2
1960-09-30,2.70,2839.022,5.6
1960-12-31,1.21,2802.616,6.3
1961-03-31,-0.40,2819.264,6.8


In [135]:
ldata['value2'] = np.random.randn(len(ldata))

In [136]:
pivoted = ldata.pivot('date', 'item')

In [138]:
pivoted[:5]

value                    value2                    
item        infl   realgdp unemp      infl   realgdp     unemp
date                                                          
1959-03-31  0.00  2710.349   5.8  1.186817 -0.494907  0.302129
1959-06-30  2.34  2778.801   5.1  0.122905 -0.854621 -2.425170
1959-09-30  2.74  2775.488   5.3  0.007938 -0.346991 -1.668020
1959-12-31  0.27  2785.204   5.6  1.965863 -0.685009  1.126503
1960-03-31  2.31  2847.699   5.2  0.104533 -1.251329 -0.422938

In [140]:
setindexed = ldata.set_index(['date', 'item'])

In [142]:
setindexed[:5]

value    value2
date       item                       
1959-03-31 realgdp  2710.349 -0.494907
           infl        0.000  1.186817
           unemp       5.800  0.302129
1959-06-30 realgdp  2778.801 -0.854621
           infl        2.340  0.122905

In [143]:
unstacked = ldata.set_index(['date', 'item']).unstack('item')

In [145]:
unstacked[:5]

value                    value2                    
item        infl   realgdp unemp      infl   realgdp     unemp
date                                                          
1959-03-31  0.00  2710.349   5.8  1.186817 -0.494907  0.302129
1959-06-30  2.34  2778.801   5.1  0.122905 -0.854621 -2.425170
1959-09-30  2.74  2775.488   5.3  0.007938 -0.346991 -1.668020
1959-12-31  0.27  2785.204   5.6  1.965863 -0.685009  1.126503
1960-03-31  2.31  2847.699   5.2  0.104533 -1.251329 -0.422938

### Pivoting “Wide” to “Long” Format
- pd.melt(data, id_vars, var_name, value_name) : 기존의 열들을 아래로 길게 쌓아 각각 value를 할당하는 방법

In [146]:
df = pd.DataFrame({'key': ['foo', 'bar', 'baz'],
 .....: 'A': [1, 2, 3],
 .....: 'B': [4, 5, 6],
 .....: 'C': [7, 8, 9]})

In [147]:
df

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


In [148]:
melted = pd.melt(df, ['key'])

In [149]:
melted

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


In [150]:
reshaped = melted.pivot('key', 'variable', 'value')

In [151]:
reshaped

variable,A,B,C
key,,,
bar,2,5,8
baz,3,6,9
foo,1,4,7


In [152]:
reshaped.reset_index()

variable,key,A,B,C
0,bar,2,5,8
1,baz,3,6,9
2,foo,1,4,7


In [153]:
pd.melt(df, id_vars=['key'], value_vars=['A', 'B'])

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6


In [154]:
pd.melt(df, value_vars=['A', 'B', 'C'])

,variable,value
0,A,1
1,A,2
2,A,3
3,B,4
4,B,5
5,B,6
6,C,7
7,C,8
8,C,9


In [155]:
pd.melt(df, value_vars=['key', 'A', 'B'])

,variable,value
0,key,foo
1,key,bar
2,key,baz
3,A,1
4,A,2
5,A,3
6,B,4
7,B,5
8,B,6
